In [1]:
!pip install cfbd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.0a2
    Uninstalling pydantic-2.11.0a2:
      Successfully uninstalled pydantic-2.11.0a2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires pydantic>=2.7.0, but you have pydantic 1.10.21 which is incompatible.
google-genai 0.2.2 requires pydantic<3.0.0dev,>=2.0.0, but you have pydantic 1.10.21 which is incompatible.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version < "3.11", but you have async-timeout 5.0.1 which is incomp

In [2]:
import cfbd
import os
import json
from datetime import datetime
from kaggle_secrets import UserSecretsClient
import pandas as pd
import time  # For adding delays

In [3]:
# Get API key from Kaggle Secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("CFBD KEY")

# Configure API client with explicit authorization header
configuration = cfbd.Configuration()

# Create API instances with explicit authorization header
players_api = cfbd.PlayersApi(
    cfbd.ApiClient(configuration, header_name='Authorization', header_value=f'Bearer {api_key}'))
metrics_api = cfbd.MetricsApi(
    cfbd.ApiClient(configuration, header_name='Authorization', header_value=f'Bearer {api_key}'))

def fetch_transfer_portal_data(year=2024):
    """Fetches transfer portal data for a given year."""
    try:
        api_response = players_api.get_transfer_portal(year=year)
        return api_response
    except cfbd.ApiException as e:
        print(f"Exception when calling PlayersApi->get_transfer_portal: {e}")
        return None


def fetch_player_season_ppa_data(year=2024, player_id=None):
    """Fetches player season PPA data for a given year and player ID."""
    try:
        if player_id is not None:
            api_response = metrics_api.get_player_season_ppa(year=year, player_id=player_id)
        else:
            api_response = metrics_api.get_player_season_ppa(year=year)
        return api_response
    except cfbd.ApiException as e:
        print(f"Exception when calling MetricsApi->get_player_season_ppa: {e}")
        return None


def fetch_player_id(first_name, last_name, origin, year=2024):
    """Fetches player ID using player search API."""
    try:
        search_term = f"{first_name} {last_name}"
        api_response = players_api.search_players(search_term, team=origin, year=year)
        if api_response and len(api_response) > 0:
            return api_response[0].id  # Return the first matching player's ID
        return None
    except cfbd.ApiException as e:
        print(f"Exception when calling PlayersApi->search_players: {e}")
        return None


def save_data_to_json(data, filename):
    """Saves data to a JSON file in Kaggle's working directory."""

    if data:

        def convert_datetime(obj):
            if isinstance(obj, datetime):
                return obj.isoformat()
            return obj

        filepath = os.path.join("/kaggle/working/", filename)
        with open(filepath, "w") as f:
            json.dump(
                [json.loads(json.dumps(obj.to_dict(), default=convert_datetime)) for obj in data],
                f,
                indent=4,
            )
        print(f"Data saved to {filepath}")


def create_transfer_dataframe(portal_data):
    """Creates a Pandas DataFrame from the transfer portal data."""
    if portal_data:
        transfers = []
        for player in portal_data:
            transfers.append({
                'firstName': player.firstName,
                'lastName': player.lastName,
                'origin': player.origin,
                'stars': player.stars,
            })
        return pd.DataFrame(transfers)
    return pd.DataFrame()  # Return an empty DataFrame if no data


def add_ppa_to_dataframe(transfer_df):
    """Adds PPA data to the transfer DataFrame."""
    ppa_data = []
    for index, row in transfer_df.iterrows():
        try:
            player_id = fetch_player_id(
                first_name=row['firstName'],
                last_name=row['lastName'],
                origin=row['origin'],
            )
            if player_id:
                time.sleep(1)  # Increase delay to 1 second
                player_ppa_data = metrics_api.get_player_season_ppa(player_id=player_id)
                if player_ppa_data and player_ppa_data:
                    # Assuming player_ppa_data is a list, take the first element if available
                    ppa = player_ppa_data[0].ppa if player_ppa_data and player_ppa_data[0].ppa else None
                    transfer_df.loc[index, 'ppa'] = ppa
            else:
                print(
                    f"No matching player found for {row['firstName']} {row['lastName']} from {row['origin']}"
                )
        except cfbd.ApiException as e:
            print(f"API Exception: {e}")
            time.sleep(10)  # Wait longer after an exception
    return transfer_df


# Fetch transfer portal data
portal_data = fetch_transfer_portal_data()

if portal_data:
    transfer_df = create_transfer_dataframe(portal_data)
    print(transfer_df.head())
    transfer_df = add_ppa_to_dataframe(transfer_df)
    print(transfer_df.head())
else:
    print("Failed to fetch transfer portal data. Exiting.")

Exception when calling PlayersApi->get_transfer_portal: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 04 Apr 2025 18:10:58 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '42', 'Connection': 'keep-alive', 'Content-Security-Policy': "default-src 'self';base-uri 'self';font-src 'self' https: data:;form-action 'self';frame-ancestors 'self';img-src 'self' data:;object-src 'none';script-src 'self';script-src-attr 'none';style-src 'self' https: 'unsafe-inline';upgrade-insecure-requests", 'Cross-Origin-Opener-Policy': 'same-origin', 'Cross-Origin-Resource-Policy': 'same-origin', 'Origin-Agent-Cluster': '?1', 'Referrer-Policy': 'no-referrer', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains', 'X-Content-Type-Options': 'nosniff', 'X-DNS-Prefetch-Control': 'off', 'X-Download-Options': 'noopen', 'X-Frame-Options': 'SAMEORIGIN', 'X-Permitted-Cross-Domain-Policies': 'none', 'X-XSS-Protection': '0', 'Access-Control-Al